In [1]:
import pandas as pd
import numpy as np

recipes = pd.read_csv('filtered_recipes.csv')

In [2]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134746 entries, 0 to 134745
Data columns (total 26 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                134746 non-null  int64  
 1   name                              134746 non-null  object 
 2   description                       134078 non-null  object 
 3   ingredients_raw                   134746 non-null  object 
 4   steps                             134746 non-null  object 
 5   servings                          134746 non-null  float64
 6   serving_size                      134746 non-null  object 
 7   tags                              134746 non-null  object 
 8   ingredients                       134746 non-null  object 
 9   amounts                           134746 non-null  object 
 10  amount_gram                       134746 non-null  object 
 11  serving_size_numeric              134746 non-null  f

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
recipes.head(1)

,id,name,description,ingredients_raw,steps,servings,serving_size,tags,ingredients,amounts,amount_gram,serving_size_numeric,predicted_total,actual_total,approx_rate,total_recipe_weight,recipe_energy_per100g,recipe_carbohydrates_per100g,recipe_proteins_per100g,recipe_fat_per100g,recipe_energy_kcal_per100g,recipe_energy_per_serving,recipe_carbohydrates_per_serving,recipe_proteins_per_serving,recipe_fat_per_serving,recipe_energy_kcal_per_serving
0,76133,Reuben and Swiss Casserole Bake,I think this is even better than a reuben sand...,"[""1/2-1 lb corned beef, cooked and choppe...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course...","['corned beef', 'thousand island dressing', 's...","[{'unit': 'pound', 'amount_min': 0.5, 'amount_...","['226.8-453.6', 60.0, 453.6, 226.8, 150.0, 53.9]",207.0,1284.5,828.0,0.644609,1284.5,837.891086,8.031474,12.64761,13.490992,200.260776,1734.434548,16.625151,26.180553,27.926353,414.539806


In [5]:
import yaml
import pandas as pd

def safe_yaml_parse(s):
    try:
        # Replace single quotes with double quotes for YAML compatibility
        s_clean = s.replace("'", '"')
        parsed = yaml.safe_load(s_clean)
        # Ensure parsed result is a list (not None or other types)
        return parsed if isinstance(parsed, list) else None
    except Exception as e:
        # Return None for failed parsing (tracked later)
        return None

# Create a new column for parsed steps (keep original 'steps')
recipes['steps_parsed'] = recipes['steps'].apply(safe_yaml_parse)

# Track validation status (True/False)
recipes['steps_validated'] = recipes['steps_parsed'].notna()

# Save failed entries with their IDs to a CSV
failed_mask = ~recipes['steps_validated']
failed_entries = recipes.loc[failed_mask, ['id', 'steps']]  # Assuming 'id' column exists
failed_entries.to_csv('failed_entries.csv', index=False)

# Optional: Drop the temporary 'steps_parsed' column if needed
# recipes = recipes.drop(columns=['steps_parsed'])

In [6]:
print(f"Successfully parsed: {recipes['steps_validated'].sum()}")
print(f"Failed: {len(recipes) - recipes['steps_validated'].sum()}")

Successfully parsed: 113775
Failed: 20971


In [7]:
import yaml

def safe_yaml_parse(s):
    try:
        # Escape existing double quotes and replace single quotes
        s_clean = s.replace('"', '\\"').replace("'", '"')
        parsed = yaml.safe_load(s_clean)
        return parsed if isinstance(parsed, list) else None  # Ensure it's a list
    except Exception as e:
        return None  # Keep as None for tracking

# Identify rows that previously failed parsing
mask = ~recipes['steps_validated']

# Reprocess only failed entries
recipes.loc[mask, 'steps_parsed'] = recipes.loc[mask, 'steps'].apply(safe_yaml_parse)

# Update validation status for reprocessed rows
recipes.loc[mask, 'steps_validated'] = recipes.loc[mask, 'steps_parsed'].notna()

# Save remaining failed entries to a new file
still_failed_mask = ~recipes['steps_validated']
still_failed_entries = recipes.loc[still_failed_mask, ['id', 'steps']]
still_failed_entries.to_csv('still_failed_entries.csv', index=False)

# Print stats
print(f"Successfully parsed after reprocessing: {recipes['steps_validated'].sum()}")
print(f"Still failed: {still_failed_entries.shape[0]}")

Successfully parsed after reprocessing: 134154
Still failed: 592


In [8]:
# Check a sample of remaining failures
still_failed_sample = recipes[still_failed_mask].sample(3)
still_failed_sample['steps'].apply(print)

["1. Stick a stick in through the top of the apple, and twirl the apple in the heated caramel dip.", "2. Roll that apple in some peanuts.", "3.  It"s called a carmel apple and ain"t it grand?"]
["Date Filling: Cook dates in water with orange rind {grated} with sugar over a moderate heat until thickened and smooth remove from heat and add fruit juices mix well and cool before spreading.", "Crumb Mixture: Sift flour, baking soda and salt rub in butter add sugar and oatmeal mix well spread 1/2 of the crumb mixture in a greased pan and press in smoothly. Cover with cooled date mixture evenly then cover with the remaining crumb mixture pat to make smooth bake 30 - 35 minutes in a cool oven 325 Fahrenheit then increase the heat to 350 Fahrenheit to lightly brown cake.", "Cut in squares while hot allow to cool in pan.", "Note: Date mixture can be cut in half if you do not want that much filling. Crumb mixture I blend the butter  well with the crumb mixture then I put the mixture in my Kitchen

18229    None
44026    None
60958    None
Name: steps, dtype: object

In [9]:
recipes = recipes[recipes['steps_validated']]
recipes = recipes.reset_index(drop=True)
print(f"Remaining rows: {len(recipes)}")


Remaining rows: 134154


In [13]:
import ast
recipes['ingredients'] = recipes['ingredients'].apply(ast.literal_eval)

In [14]:
recipes['ingredients_raw'] = recipes['ingredients_raw'].apply(ast.literal_eval)

In [15]:
recipes['amount_gram'] = recipes['amount_gram'].apply(ast.literal_eval)

In [16]:
recipes['amounts'] = recipes['amounts'].apply(ast.literal_eval)

In [17]:
recipes['tags'] = recipes['tags'].apply(ast.literal_eval)

In [16]:
recipes.to_csv('parsed_recipes.csv',index=False)

In [12]:
recipes = pd.read_csv('parsed_recipes.csv')

In [9]:
# We are going to generate questions for each recipe's description and name. 
# This will help to build our baseline model. 
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key = api_key)
# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [18]:
# We will load the full dataset and combine the name and description fields into a single text field for each recipe.
# Using a SentenceTransformer , we will convert all combined texts into embeddings.
# We will normalize these embeddings so that cosine similatirty can be computed using inner products.
# Then , we will cluster them using Faiss. Faiss's K-Means will be used to cluster the 130k+ embeddings.
# For each of the 100 centroids , we will generate 1 synthetic question and answer that cover the diversity of our dataset. 

from sentence_transformers import SentenceTransformer
import faiss

df = recipes[['name','description']].copy()


In [19]:
df["description"] = df["description"].fillna("")  
df["name_description"] = df["name"] + " - " + df["description"]
df["name_description"] = df["name_description"].str.strip(" -")  

In [20]:
# Load a SentenceTransformer model for embedding generation
embedder = SentenceTransformer('all-MiniLM-L6-v2')
# Generate embeddings for all combined texts
embeddings = embedder.encode(df["name_description"].tolist(), convert_to_numpy=True)
# Normalize embeddings so that inner product equals cosine similarity
faiss.normalize_L2(embeddings)

In [21]:
# Set the number of clusters to 1000
num_clusters = 100
d = embeddings.shape[1]

# Perform K-Means clustering with Faiss
kmeans = faiss.Kmeans(d, num_clusters, niter=20, verbose=True)
kmeans.train(embeddings)

13619.9892578125

In [22]:
# Create FAISS index for retrieving nearest neighbors to centroids.
index = faiss.IndexFlatL2(d)
index.add(embeddings)
# For each centroid, retrieve the nearest neighbor indice.
_, cluster_indices = index.search(kmeans.centroids, 1)

In [ ]:
def generate_question_and_answer(recipe_text):
    """
    Given a recipe text (combined name and description), generate:
      - A general question a user might ask (without having seen the recipe)
      - A concise answer summarizing the recipe (including the recipe name and description)
    
    The prompt instructs the model to output in a structured format:
    
    Question: <your generated general question>
    Answer: <your generated answer>
    """
    prompt = (
         "Based on the following recipe details, generate one broad and general question that a user might ask when looking for a recipe. "
        "Ignore very specific details and focus on the main category, key ingredients, or overall theme of the recipe. For example, instead of mentioning every detail, "
        "the question could be like 'Can you suggest some comfort food desserts?' or 'Are there any budget-friendly recipes that include a specific ingredient?'\n\n"
        "Also, generate a concise answer summarizing the recipe, including the recipe name and a brief description.\n\n"
        f"Recipe Details: {recipe_text}\n\n"
        "Please format your response exactly as follows:\n"
        "Question: <your general question>\n"
        "Answer: <your summary answer>"
    )
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  
        messages=[
            {"role": "system", "content": "You are a helpful culinary assistant skilled in both understanding recipe details and generalizing user search queries."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.35,
    )
    content = response.choices[0].message.content.strip()
    
    # Parse the output assuming it follows the requested format.
    # We'll split on "Question:" and "Answer:".
    question = ""
    answer = ""
    if "Question:" in content and "Answer:" in content:
        parts = content.split("Answer:")
        question_part = parts[0].split("Question:")[-1].strip()
        answer = parts[1].strip()
        question = question_part
    else:
        # Fallback: return the full content as question and empty answer.
        question = content
        answer = ""
    
    # Final clean up: remove any stray numbering or punctuation.
    question = question.strip(" -0123456789").strip()
    answer = answer.strip(" -0123456789").strip()
    return question, answer

In [ ]:
all_cluster_data = []

for cluster_id in range(num_clusters):
    rep_idx = int(cluster_indices[cluster_id][0])  # Representative index for the cluster
    recipe_text = df.iloc[rep_idx]["name_description"]
    
    # Generate one general question and an answer from the representative recipe text.
    q, a = generate_question_and_answer(recipe_text)
    
    all_cluster_data.append({
        "cluster_id": cluster_id,
        "representative_index": rep_idx,
        "recipe_text": recipe_text,
        "generated_question": q,
        "generated_answer": a
    })

In [ ]:
questions_df = pd.DataFrame(all_cluster_data)
questions_df.to_csv("cluster_questions.csv", index=False)

In [23]:
questions_df = pd.read_csv('cluster_questions.csv')

In [29]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
questions_df.head(5)

,cluster_id,representative_index,recipe_text,generated_question,generated_answer
0,0,63064,Bread Pudding - This is my mom's bread pudding recipe that she found in a magazine shortly after she and my dad were married. It is a wonderful comfort food for cool fall and winter days!,Can you recommend a comforting dessert for fall and winter days?,"Recipe Name: Mom's Bread Pudding\nDescription: A comforting bread pudding recipe perfect for cool fall and winter days, passed down from the user's mom."
1,1,54901,"Pizza Dough - This makes an awesome pizza crust, crispy and yummy",Can you recommend a classic recipe for homemade pizza dough?,"Recipe Name: Homemade Pizza Dough\nDescription: This recipe provides a simple and delicious homemade pizza dough that results in a crispy and tasty crust, perfect for making your own pizzas at home."
2,2,57229,Baked Pork Chops - This is a great meal but a little longer than your weekday meal might take. Great flavor!,Can you recommend a flavorful and slightly time-consuming dinner recipe?,Recipe Name: Baked Pork Chops\nDescription: This Baked Pork Chops recipe is a delicious and flavorful meal that takes a bit longer to prepare than a typical weekday dinner. It offers great taste and is worth the extra time spent in the kitchen.
3,3,119890,Inexpensive Caramels - This is from my Mom's recipe cards. I have not tried this recipe.,Do you have a simple recipe for homemade candy?,Recipe Name: Inexpensive Caramels\nDescription: This recipe is a classic homemade caramel recipe passed down from the user's Mom. It is a simple and inexpensive way to make delicious caramels at home.
4,4,101570,Turkey Tenderloins With Caramelized Onions - I love this with mashed potatoes! This is also good with chicken in place of turkey. It is adapted from a Betty Crocker cookbook.,Can you recommend a recipe featuring turkey tenderloins as the main ingredient?,"Recipe Name: Turkey Tenderloins With Caramelized Onions - A delicious dish featuring turkey tenderloins and caramelized onions, perfect when served with mashed potatoes. It can also be made with chicken as a substitute. Adapted from a Betty Crocker cookbook."


In [ ]:

def direct_faiss_search(query_text, top_k=5):
    """
    1) Convert query_text into an embedding with the same model that produced 'embeddings'.
    2) Normalize it (because the index presumably expects normalized embeddings).
    3) Search your FAISS index for the top_k hits.
    4) Return a DataFrame slice of the best matches.
    """
    # 1) Encode the query
    q_emb = embedder.encode([query_text], convert_to_numpy=True)
    
    # 2) Normalize so that L2 distance = 2 - 2*cosine_similarity
    faiss.normalize_L2(q_emb)
    
    # 3) Query FAISS
    distances, indices = index.search(q_emb, top_k)
    
    # 4) Return the matching rows from df
    # 'indices' is shape (1, top_k)
    best_idx = indices[0]
    return df.iloc[best_idx]

# Now let's test it with your question:
question = questions_df.loc[2,'generated_question']
results_df = direct_faiss_search(question, top_k=5)

print("QUESTION:", question)
print("\nFAISS Search Results:\n", results_df)


QUESTION: Can you recommend a flavorful and slightly time-consuming dinner recipe?

FAISS Search Results:
                                        name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   description  \
86630                  International Dinner                                                                                                                                                                                                                                                                     

In [171]:
print(questions_df.loc[2,'recipe_text'])

Baked Pork Chops - This is a great meal but a little longer than your weekday meal might take. Great flavor!


In [60]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "Qwen/Qwen2.5-0.5B-Instruct"  
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
qwen_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
qwen_model.eval()

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

c:\Users\turgu\anaconda3\envs\llms\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\turgu\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Qwen model is running on: {device.upper()}")

qwen_model.to(device)

Qwen model is running on: CUDA


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [151]:
import re
import json

def extract_final_json_block(raw_text):
    """
    1) Find all substrings that start with '{', contain "requested_count",
       and continue until the first matching '}'.
    2) Return the LAST one found, in case Qwen prints multiple partial blocks or instructions.
    3) Attempt to parse it as JSON; if parse fails, fallback to out-of-scope.

    This approach ignores any lines with 'assistant' or system instructions,
    because it hunts only for the final curly brace block containing "requested_count".
    """

    # This regex does a LAZY match of any characters (including newlines) from 
    # the literal '{' up through the first '}' that closes the block containing "requested_count".
    pattern = r'\{\s*"requested_count"(?:.|\n)*?\}'

    matches = re.findall(pattern, raw_text)
    if not matches:
        # If no match, we have no valid JSON => fallback
        return {
            "requested_count": 0,
            "search_query": "[Out of scope or needs clarification]"
        }

    # Take the last match to get the final JSON block
    last_block = matches[-1].strip()

    try:
        parsed = json.loads(last_block)
        return parsed
    except Exception as e:
        print("[DEBUG] parse error on last_block:", e)
        return {
            "requested_count": 0,
            "search_query": "[Out of scope or needs clarification]"
        }

        

def rephrase_query_for_retrieval_json(original_query):
    

    system_instructions = (
    "You are a helpful assistant that extracts structured recipe request info from user input.\n"
    "Return ONLY a valid JSON object like this:\n"
    '{ "requested_count": <integer>, "search_query": "<keywords or constraints>" }\n\n'
    "Rules:\n"
    "- If user says 'Give me X recipes' or 'I want X dishes', set requested_count = X , otherwise set requested_count = 5.\n"
    "- Time or ingredient limits (e.g. '3 ingredients', '15 minutes') go in search_query.\n"
    "- If unclear or out of scope, respond with:\n"
    '{ \"requested_count\": 0, \"search_query\": \"[Out of scope or needs clarification]\" }\n'
    "- No other output. Only JSON. No text or role names.\n\n"
    "Examples:\n"
    "\"Give me 3 chicken recipes\"\n"
    "→ { \"requested_count\": 3, \"search_query\": \"chicken\" }\n\n"
    "\"Quick vegan meals\"\n"
    "→ { \"requested_count\": 5, \"search_query\": \"quick vegan meals\" }\n\n"
    "\"I want 10 dishes with at most 3 ingredients, ready in 15 minutes\"\n"
    "→ { \"requested_count\": 10, \"search_query\": \"at most 3 ingredients, 15 minutes\" }\n\n"
    "\"I want some recipes with tofu\"\n"
    "→ { \"requested_count\": 5, \"search_query\": \"tofu\" }"
    "\"Give me some recipes with chicken as a main ingredient.\"\n"
    "→ { \"requested_count\": 5, \"search_query\": \"chicken\" }"
)



    user_message = f"{original_query}\n→"


    messages = [
        {"role": "system", "content": system_instructions},
        {"role": "user", "content": user_message}
    ]

    # Build final prompt for Qwen
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    print("\n===== [DEBUG] Final Prompt =====")
    print(prompt)
    print("================================\n")

    inputs = tokenizer(prompt, return_tensors="pt").to(qwen_model.device)
    with torch.no_grad():
        output = qwen_model.generate(**inputs, max_new_tokens=150)

    raw_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    print("===== [DEBUG] Raw Qwen Output =====")
    print(raw_text)
    print("===================================\n")

    # Use the regex-based extraction
    parsed = extract_final_json_block(raw_text)

    print("===== [DEBUG] Parsed JSON =====")
    print(parsed)
    print("================================\n")

    return parsed




In [176]:
user_question = questions_df.loc[5,'generated_question']
parsed_result = rephrase_query_for_retrieval_json(user_question)

if parsed_result["requested_count"] == 0 and parsed_result["search_query"].startswith("[Out of scope"):
    print("Qwen flagged the query or parse error =>", parsed_result["search_query"])
else:
    print("Final parsed JSON =>", parsed_result)

print(questions_df.loc[5,'recipe_text'])
direct_faiss_search(parsed_result["search_query"],5)




===== [DEBUG] Final Prompt =====
<|im_start|>system
You are a helpful assistant that extracts structured recipe request info from user input.
Return ONLY a valid JSON object like this:
{ "requested_count": <integer>, "search_query": "<keywords or constraints>" }

Rules:
- If user says 'Give me X recipes' or 'I want X dishes', set requested_count = X , otherwise set requested_count = 5.
- Time or ingredient limits (e.g. '3 ingredients', '15 minutes') go in search_query.
- If unclear or out of scope, respond with:
{ "requested_count": 0, "search_query": "[Out of scope or needs clarification]" }
- No other output. Only JSON. No text or role names.

Examples:
"Give me 3 chicken recipes"
→ { "requested_count": 3, "search_query": "chicken" }

"Quick vegan meals"
→ { "requested_count": 5, "search_query": "quick vegan meals" }

"I want 10 dishes with at most 3 ingredients, ready in 15 minutes"
→ { "requested_count": 10, "search_query": "at most 3 ingredients, 15 minutes" }

"I want some recip

,name,description,name_description
14471,Joanne's Sweet and Hot Pickles,"This recipe was given to me by a grand lady that I work with. It is very easy, fast and taste very good.","Joanne's Sweet and Hot Pickles - This recipe was given to me by a grand lady that I work with. It is very easy, fast and taste very good."
25924,Simply Sweet Pickles (No Processing Required),"This is the simplest recipe ever! I've canned a lot of pickles, and this is now our favorite sweet pickle recipe. And the beauty of it is there is no water bath required! It has a little kick to it from tobasco sauce. I do NOT like hot, spicey food so I was sceptical at first, but trust me, there is just enough kick to this to make it interesting.","Simply Sweet Pickles (No Processing Required) - This is the simplest recipe ever! I've canned a lot of pickles, and this is now our favorite sweet pickle recipe. And the beauty of it is there is no water bath required! It has a little kick to it from tobasco sauce. I do NOT like hot, spicey food so I was sceptical at first, but trust me, there is just enough kick to this to make it interesting."
58545,Crisp Sweet Pickles,Super easy and they taste sooo good! I love this recipe because it only takes 10 minutes + chilling time to make. I found this recipe in Taste of Home magazine and now I keep it on hand in my recipe box!,Crisp Sweet Pickles - Super easy and they taste sooo good! I love this recipe because it only takes 10 minutes + chilling time to make. I found this recipe in Taste of Home magazine and now I keep it on hand in my recipe box!
68444,Sweet Hot Pickles,"This is a combination of several recipes, one from my father, one from an old cook book, and my own adjustments","Sweet Hot Pickles - This is a combination of several recipes, one from my father, one from an old cook book, and my own adjustments"
98261,Lite Spicy Dill Pickle Dip,"Make with your favorite dill pickle, or better yet, your homemade ones","Lite Spicy Dill Pickle Dip - Make with your favorite dill pickle, or better yet, your homemade ones"
